In [1]:
import psycopg2
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot 
import plotly.graph_objects as go
import warnings
import datetime
warnings.filterwarnings('ignore')

# import q_credentials.db_secmaster_cloud_cred as db_secmaster_cred
import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred


In [2]:
# connect to our securities_master database
# conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)

In [3]:
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

In [6]:
# sql="""SELECT s.ticker, max(d.symbol_id), max(d.date_price)
# FROM public.symbol s inner join d_data d on s.id= d.symbol_id
# group by s.ticker"""
# df_ticker_last_day=pd.read_sql(sql,con=conn_secmaster)

In [7]:
# df_ticker_last_day

In [8]:
# sql="""select date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data where stock_id=71"""
# df_stock=pd.read_sql(sql,con=conn_secmaster)

In [9]:
# import datetime

In [10]:
sql="select * from indicator"
ind_list=list(pd.read_sql(sql,con=conn_indicator)['name'])
ind_list

['candle_1', 'candle_2', 'candle_3', 'anomaly', 'level', '', 'alpa_ind']

In [11]:
symbol_id='AAPL'#'BOM500114'
start_date=datetime.datetime(2018,1,1).strftime("%Y-%m-%d")
# indicator_name = 'candle_1'

In [12]:
# sql="select d.date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data d join symbol s on d.stock_id = s.id where s.ticker='%s' and d.date_price > '%s'" %(symbol_id, start_date)
# df_price=pd.read_sql(sql,con=conn_secmaster)

In [13]:
# sql="select d.date_price as date, d.value from daily_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, indicator_name, start_date)
# df_indicator=pd.read_sql(sql,con=conn_indicator)

In [14]:
# df_indicator.set_index('date',inplace=True)

In [15]:
# df_indicator.head()

In [16]:
pd.options.display.max_columns = None

In [17]:
df_all_ind=pd.DataFrame()
for ind in ind_list:
    print(ind)
    sql="select d.date_price as date, d.value from d_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, ind, start_date)
    df_indicator=pd.read_sql(sql,con=conn_indicator)
    if df_indicator.empty:
        continue
    df_indicator.set_index('date',inplace=True)
    df_indicator=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)
    df_indicator.columns=[ind+"_"+col for col in df_indicator.columns]
    if df_all_ind.empty:
#         df_all_ind=pd.merge(left=df_price, right=df_indicator,on='date')
        df_all_ind=df_indicator
    else:
        df_all_ind=pd.merge(left=df_all_ind, right=df_indicator,on='date')

candle_1
candle_2
candle_3
anomaly
level

alpa_ind


In [21]:
df_all_ind.reset_index(inplace=True)
df_all_ind.rename(columns={'anomaly_close':'close','anomaly_low':'low','anomaly_high':'high','anomaly_open':'open','anomaly_volume':'volume'},inplace=True)

In [22]:
# df_level=df_all_ind[['date','high','low']]
# df_level['res_outer']=df_all_ind['level_nearest_resistance_range'].str[0]
# df_level['res_inner']=df_all_ind['level_nearest_resistance_range'].str[1]
# df_level['sup_outer']=df_all_ind['level_nearest_support_range'].str[0]
# df_level['sup_inner']=df_all_ind['level_nearest_support_range'].str[1]
# df_level['target']=''
# # df_level['target_support']=0

In [31]:
df_all_ind['candle_2_pattern_name'].unique()

array(['', 'BULLISH (Matching Low)', 'BULLISH HARAMI CROSS',
       'BEARISH ENGULFING', 'BEARISH (Dark Cloud Cover)',
       'BULLISH HARAMI', 'BEARISH HARAMI CROSS', 'BULLISH ENGULFING',
       'BEARISH HARAMI  ', 'BULLISH (Piercing Pattern)',
       'BEARISH (Meeting Lines)'], dtype=object)

In [ ]:
for i, row in df_all_ind.iterrows():
    if row['candle_2_pattern_name']=='BULLISH ENGULFING':
        sl=row['close']-row['low']
        tp=2*row['close']
    elif row['candle_2_pattern_name']=='BEARISH ENGULFING':

In [ ]:
'BEARISH ENGULFING'

In [24]:
# df_level.head()

In [50]:
# Taking a forward look into the future and looking if the levels have been broken
for i, row in df_level.iterrows():
    for j in range(i+1,len(df_level)):
        if df_level.iloc[j]['high']>=df_level.iloc[i]['res_inner']:
            df_level.at[i,'target']='Resistance'
            break
        if df_level.iloc[j]['low']<=df_level.iloc[i]['sup_inner']:
            df_level.at[i,'target']='Support'
            break

In [34]:
# len(df_level[df_level['target_resistance']==1])
# len(df_level[df_level['target_support']==1])
# len(df_level[(df_level['target_support']==0) & (df_level['target_resistance']==0)])

1

In [39]:
df_level.head()

,date,high,low,res_outer,res_inner,sup_outer,sup_inner,target
0,2018-01-02,43.0750,42.3150,43.5875,43.3125,41.3200,42.3700,-1
1,2018-03-14,45.1300,44.4525,NaN,NaN,44.4775,44.5625,1
2,2018-03-15,45.0600,44.5175,45.8750,45.6475,44.4775,44.5625,1
3,2018-03-16,44.7800,44.4050,45.8750,45.6475,44.4775,44.5625,1
4,2018-03-19,44.3675,43.4150,45.8750,45.6475,43.5675,43.7350,1


In [51]:
df_feature_target=pd.merge(left=df_level[df_level.columns.difference(['low', 'high'])], right=df_all_ind,on='date')

In [52]:
columns=['date','target', 
'candle_1_pattern_name','candle_2_pattern_name', 'candle_3_pattern_name',
'level_nearest_support_atr_range','level_nearest_support_duration','level_nearest_resistance_atr_range','level_nearest_resistance_duration',
'anomaly_gap_signal', 'anomaly_vol_anomaly', 'anomaly_vol_anomaly_2','anomaly_wick_bear_signal','anomaly_wick_bull_signal']

In [57]:
columns=['date','target', 
'candle_1_pattern_name','candle_2_pattern_name', 'candle_3_pattern_name','level_nearest_support_duration','level_nearest_resistance_duration',
'anomaly_gap_signal', 'anomaly_vol_anomaly', 'anomaly_vol_anomaly_2','anomaly_wick_bear_signal','anomaly_wick_bull_signal']

In [58]:
df_feature_target[columns].to_csv('feature_target.csv',index=False)

In [59]:
df_feature_target[columns]

,date,target,candle_1_pattern_name,candle_2_pattern_name,candle_3_pattern_name,level_nearest_support_duration,level_nearest_resistance_duration,anomaly_gap_signal,anomaly_vol_anomaly,anomaly_vol_anomaly_2,anomaly_wick_bear_signal,anomaly_wick_bull_signal
0,2018-01-02,Resistance,long,,,61,13.0,0.008986,1.204958,0.0,0.760492,0.000000
1,2018-03-14,Support,,BULLISH (Matching Low),,8,NaN,0.000000,0.027358,0.0,0.132932,0.000000
2,2018-03-15,Support,DOJI_SPINOFF,BULLISH HARAMI CROSS,,9,2.0,0.000000,0.676625,0.0,0.000000,1.105611
3,2018-03-16,Support,short,BEARISH ENGULFING,,10,3.0,0.000000,0.411342,0.0,0.000000,0.000000
4,2018-03-19,Support,,,,12,6.0,-0.226364,0.351835,0.0,1.295017,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
714,2018-03-07,Resistance,DOJI_SPINOFF,,,5,48.0,-0.484049,0.853939,0.0,0.054158,0.003319
715,2018-03-08,Resistance,,,BEARISH Abandoned Baby,6,2.0,0.000000,0.064665,1.0,0.000000,0.000000
716,2018-03-09,Resistance,,,,3,50.0,0.000000,0.198207,0.0,0.000000,0.000000
717,2018-03-12,Support,,,BULLISH Three White Soldiers,53,NaN,0.000000,0.122942,0.0,0.000000,0.127307
